<img height="60px" src="https://colab.research.google.com/img/colab_favicon.ico" align="left" hspace="20px" vspace="5px">
# How to Use The Pre-Trained VGG Model to Classify Objects in Photographs

Convolutional neural networks are now capable of outperforming humans on some computer vision tasks, such as classifying images.

That is, given a photograph of an object, answer the question as to which of 1,000 specific objects the photograph shows.

A competition-winning model for this task is the VGG model by researchers at Oxford. What is important about this model, besides its capability of classifying objects in photographs, is that the model weights are freely available and can be loaded and used in your own models and applications.

In this tutorial, you will discover the VGG convolutional neural network models for image classification.

After completing this tutorial, you will know:

About the ImageNet dataset and competition and the VGG winning models.
How to load the VGG model in Keras and summarize its structure.
How to use the loaded VGG model to classifying objects in ad hoc photographs.

* Reference: https://machinelearningmastery.com/use-pre-trained-vgg-model-classify-objects-photographs/

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/learning-stack/Colab-ML-Playbook/blob/master/Convolutional%20Neural%20Network/How%20to%20Use%20The%20Pre-Trained%20VGG%20Model%20to%20Classify%20Objects%20in%20Photographs.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/learning-stack/Colab-ML-Playbook/blob/master/Convolutional%20Neural%20Network/How%20to%20Use%20The%20Pre-Trained%20VGG%20Model%20to%20Classify%20Objects%20in%20Photographs.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Load the VGG Model in Keras

In [0]:
from keras.applications.vgg16 import VGG16
model = VGG16()

In [0]:
from keras.applications.vgg16 import VGG16
model = VGG16()
print(model.summary())

In [0]:
from keras.applications.vgg16 import VGG16
from keras.utils.vis_utils import plot_model
model = VGG16()
plot_model(model, to_file='vgg.png')

## Develop a Simple Photo Classifier

In [0]:
# Upload a sample photo

# Source is from: https://www.flickr.com/photos/jfanaian/4994221690/

from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
# Load the VGG Model

from keras.applications.vgg16 import VGG16
# load the model
model = VGG16()

In [0]:
# Load and Prepare Image

from keras.preprocessing.image import load_img
# load an image from file
image = load_img('mug.jpg', target_size=(224, 224))

In [0]:
from keras.preprocessing.image import img_to_array
# convert the image pixels to a numpy array
image = img_to_array(image)

The network expects one or more images as input; that means the input array will need to be 4-dimensional: samples, rows, columns, and channels.

We only have one sample (one image). We can reshape the array by calling reshape() and adding the extra dimension.

In [0]:
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

Next, the image pixels need to be prepared in the same way as the ImageNet training data was prepared. Specifically, from the paper:

"The only preprocessing we do is subtracting the mean RGB value, computed on the training set, from each pixel."

Keras provides a function called preprocess_input() to prepare new input for the network.

In [0]:
from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image = preprocess_input(image)

In [0]:
# Make a Prediction

# predict the probability across all output classes
yhat = model.predict(image)

Keras provides a function to interpret the probabilities called decode_predictions().

It can return a list of classes and their probabilities in case you would like to present the top 3 objects that may be in the photo.

In [0]:
# Interpret Prediction

from keras.applications.vgg16 import decode_predictions
# convert the probabilities to class labels
label = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))